# ProteinTTT — Google Colab
### Customize ESMFold with Test-Time Training for Enhanced Structure Prediction

📄 [Paper](https://arxiv.org/abs/2411.02109) | 💻 [GitHub](https://github.com/anton-bushuiev/ProteinTTT)

**How to use:** Edit the **sequence & config** cell below, then **Runtime → Run all**. Results appear at the bottom.

> ⚠️ Go to **Runtime → Change runtime type → GPU** (T4 or better) before running.

---
## ✏️ Input — edit this cell

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  EDIT YOUR SEQUENCE AND SETTINGS HERE, THEN "Runtime → Run all"           ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

SEQUENCE = (
    "GIHLGELGLLPSTVLAIGYFENLVNIICESLNMLPKLEVSGKEYKKFKFTIVIPKDLDANIKKRAKIYFKQKSLIEIEIPTSSRNYPIHIQFDENSTDDILHLYDMPTTIGGIDKAIEMFMRKGHIGKTDQQKLLEERELRNFKTTLENLIATDAFAKEMVEVIIEE"
)

# ProteinTTT hyper-parameters
TTT_STEPS     = 5     # number of TTT optimisation steps  (5–30)
LEARNING_RATE = 4e-4   # step size for parameter updates
BATCH_SIZE    = 4      # samples per TTT batch
LORA_RANK     = 8      # rank of LoRA adapter matrices
LORA_ALPHA    = 32     # scaling factor for LoRA

# Where to save PDB files (Colab /content is writable)
OUTPUT_DIR = "/content/proteinttt_results"

---
*Everything below runs automatically — no edits needed.*
## 1 · Check GPU & validate input

In [2]:
import torch, os

# GPU check
if not torch.cuda.is_available():
    raise RuntimeError("No GPU detected! Go to Runtime → Change runtime type → GPU.")
gpu_name = torch.cuda.get_device_name(0)
gpu_mem  = torch.cuda.get_device_properties(0).total_memory / 1024**3
DEVICE   = "cuda"
print(f"✅ GPU: {gpu_name} ({gpu_mem:.1f} GB)")

# Validate sequence
_seq = SEQUENCE.strip().upper()
_valid_aa = set("ACDEFGHIKLMNPQRSTVWY")
assert len(_seq) >= 10,                        f"Sequence too short ({len(_seq)} aa). Need ≥ 10."
assert all(c in _valid_aa for c in _seq),       "Sequence contains invalid characters. Use standard 20 amino acids."
assert len(_seq) <= 400,                        f"Sequence too long ({len(_seq)} aa). Max 400 for this demo."
SEQUENCE = _seq
print(f"✅ Sequence accepted: {len(SEQUENCE)} residues")

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✅ Output directory: {OUTPUT_DIR}")

✅ GPU: Tesla T4 (14.6 GB)
✅ Sequence accepted: 167 residues
✅ Output directory: /content/proteinttt_results


## 2 · Install dependencies (takes ~5 min on first run)

In [3]:
# ── Pre-load numpy C extensions BEFORE pip can change files on disk. ──────────
# numpy lazy-loads submodules (random, fft, linalg). If pip replaces numpy
# on disk mid-session, the lazily-loaded .so files won't match the already-
# loaded core → ABI crash.  Eagerly importing them here locks everything in
# memory from the SAME consistent build.
import numpy, numpy.random, numpy.fft, numpy.linalg
_NUMPY_VER = numpy.__version__
print(f"  ✅ numpy {_NUMPY_VER} C extensions locked in memory")

# ── Pin numpy to the EXACT in-memory version so pip cannot change it ─────────
!pip install -q \
    "numpy=={_NUMPY_VER}" \
    biopython==1.85 \
    fair-esm omegaconf==2.3.0 \
    biotite==0.40.0 modelcif einops==0.8.1 \
    ml-collections dm-tree \
    "pytorch-lightning>=1.8.4,<2.5" \
    "torchmetrics>=0.10.3" \
    "transformers>=4.30.0" \
    "diffusers>=0.15.0"

# Install proteinttt WITHOUT pulling its transitive deps (avoids numpy change)
!pip install -q --no-deps "proteinttt @ git+https://github.com/anton-bushuiev/ProteinTTT.git@e218659626214cc9aba302675f4285bb2ba6332c"

# ── Patch fair-esm for Python 3.12 (mutable dataclass defaults) ──
import importlib, importlib.util, re, sys, os

import glob as _glob
_esm_spec = importlib.util.find_spec("esm")
if _esm_spec and _esm_spec.origin:
    _esm_v1_dir = os.path.join(
        os.path.dirname(_esm_spec.origin), "esmfold", "v1"
    )
    # Patch ALL .py files under esm/esmfold/v1/ that have mutable dataclass defaults
    for _fp in _glob.glob(os.path.join(_esm_v1_dir, "*.py")):
        with open(_fp) as _f:
            _src = _f.read()
        if "default_factory" in _src or "@dataclass" not in _src:
            continue  # already patched or no dataclasses to fix
        print(f"⏳ Patching {_fp} for Python 3.12 dataclass compatibility…")
        # Add 'from dataclasses import field' if missing
        if "from dataclasses import field" not in _src:
            _src = _src.replace(
                "from dataclasses import dataclass",
                "from dataclasses import dataclass, field",
            )
            # If file uses dataclass but doesn't import it directly (uses full path)
            if "from dataclasses import" not in _src:
                _src = "from dataclasses import field\n" + _src
        # Replace mutable defaults:  = SomeConfig()  →  = field(default_factory=SomeConfig)
        # Handles both `: SomeConfig = SomeConfig()` and `: T.Any = SomeConfig()` patterns
        _src = re.sub(
            r"(:\s*\S+)\s*=\s*(\w+Config)\(\)",
            r"\1 = field(default_factory=\2)",
            _src,
        )
        with open(_fp, "w") as _f:
            _f.write(_src)
        print(f"  ✅ Patched {os.path.basename(_fp)}")
    print("  ✅ esm/esmfold/v1/ dataclass patches complete.")
else:
    print("  ⚠️ esm package not found — skipping dataclass patches")

# Verify critical packages imported correctly
import pytorch_lightning, torchmetrics, transformers
print(f"✅ pip packages installed "
      f"(pytorch-lightning {pytorch_lightning.__version__}, "
      f"transformers {transformers.__version__}).")

  ✅ numpy 2.0.2 C extensions locked in memory
ERROR: Cannot install biopython==1.85, biotite==0.40.0 and numpy==2.0.2 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  ⚠️ esm package not found — skipping dataclass patches


ModuleNotFoundError: No module named 'pytorch_lightning'

## 3 · Install openfold & lora-diffusion (patched builds)

In [ ]:
import importlib, importlib.util, os, re, shutil, subprocess, sys, tempfile

def _pip_install(*args):
    cmd = [sys.executable, "-m", "pip", "install", *args]
    print(f"  → {' '.join(cmd)}")
    subprocess.check_call(cmd)
    importlib.invalidate_caches()

# ── openfold ──
try:
    import openfold
    print("✅ openfold already installed.")
except ImportError:
    print("⏳ Installing openfold (may take a few minutes)…")
    _pip_install("--no-build-isolation",
                 "dllogger @ git+https://github.com/NVIDIA/dllogger.git")
    _of_dir = os.path.join(tempfile.gettempdir(), "_openfold_build")
    if os.path.exists(_of_dir):
        shutil.rmtree(_of_dir)
    subprocess.check_call(["git", "clone", "--filter=blob:none", "-q",
                           "https://github.com/aqlaboratory/openfold.git", _of_dir])
    subprocess.check_call(["git", "checkout", "-q",
                           "4b41059694619831a7db195b7e0988fc4ff3a307"], cwd=_of_dir)
    _sp = os.path.join(_of_dir, "setup.py")
    with open(_sp) as f:
        _src = f.read()
    _src = _src.replace("from scripts.utils import get_nvidia_cc",
                         "# from scripts.utils import get_nvidia_cc  # patched")
    _src = re.sub(r"compute_capabilities\s*=\s*set\(\[.*?set\(\[compute_capability\]\)",
                  "compute_capabilities = set([(7, 0), (8, 0), (8, 6), (9, 0)])",
                  _src, flags=re.DOTALL)
    _src = _src.replace("-std=c++14", "-std=c++17")
    with open(_sp, "w") as f:
        f.write(_src)
    os.environ["TORCH_CUDA_ARCH_LIST"] = "7.0;8.0;8.6;9.0"
    _pip_install("--no-build-isolation", _of_dir)
    shutil.rmtree(_of_dir, ignore_errors=True)

    # ── Patch openfold for pytorch-lightning ≥ 2.0 ──
    # seed_everything moved from pytorch_lightning.utilities.seed → top-level
    _of_seed = os.path.join(
        os.path.dirname(importlib.util.find_spec("openfold").origin),
        "utils", "seed.py",
    )
    if os.path.isfile(_of_seed):
        with open(_of_seed) as _f:
            _seed_src = _f.read()
        if "pytorch_lightning.utilities.seed" in _seed_src:
            _seed_src = _seed_src.replace(
                "from pytorch_lightning.utilities.seed import seed_everything",
                "from pytorch_lightning import seed_everything",
            )
            with open(_of_seed, "w") as _f:
                _f.write(_seed_src)
            print("  ✅ Patched openfold/utils/seed.py for pytorch-lightning ≥ 2.0")

    import openfold
    print("✅ openfold installed.")

# ── lora-diffusion ──
if importlib.util.find_spec("lora_diffusion") is None:
    print("⏳ Installing lora-diffusion…")
    _pip_install("--no-deps", "--no-build-isolation",
                 "lora-diffusion @ git+https://github.com/cloneofsimo/lora.git")
    _spec = importlib.util.find_spec("lora_diffusion")
    _init = os.path.join(os.path.dirname(_spec.origin), "__init__.py")
    with open(_init, "w") as f:
        f.write("from .lora import *\n")
    from lora_diffusion.lora import inject_trainable_lora as _chk
    del _chk
    print("✅ lora-diffusion installed.")
else:
    print("✅ lora-diffusion already installed.")

✅ openfold already installed.
✅ lora-diffusion already installed.


## 4 · Load ESMFold model onto GPU

In [ ]:
import sys, importlib, glob, re, torch

# ── Patch ALL esm dataclass files for Python 3.12 ──────────────────────────
# Python 3.12 rejects mutable defaults in dataclasses; fair-esm uses them
# extensively. We patch every .py under esm/esmfold/ that has this pattern.

_esm_roots = glob.glob(f"{sys.prefix}/lib/python*/dist-packages/esm/esmfold") + \
             glob.glob(f"{sys.prefix}/lib/python*/site-packages/esm/esmfold")

_patched = 0
for _root in _esm_roots:
    import os
    for _dirpath, _, _fnames in os.walk(_root):
        for _fname in _fnames:
            if not _fname.endswith(".py"):
                continue
            _fp = os.path.join(_dirpath, _fname)
            with open(_fp) as _f:
                _src = _f.read()

            _new = _src
            # Ensure 'field' is imported from dataclasses
            if "from dataclasses import dataclass" in _new and "field" not in _new:
                _new = _new.replace(
                    "from dataclasses import dataclass",
                    "from dataclasses import dataclass, field",
                )
            # Replace   : SomeType = SomeType(...)   →   : SomeType = field(default_factory=SomeType)
            # Handles zero-arg and any-arg constructors
            _new = re.sub(
                r"(:\s*)(\w+)\s*=\s*\2\([^)]*\)",
                r"\1\2 = field(default_factory=\2)",
                _new,
            )

            if _new != _src:
                with open(_fp, "w") as _f:
                    _f.write(_new)
                _patched += 1
                print(f"  Patched: {_fp}")

if _patched:
    print(f"✅ Patched {_patched} file(s) for Python 3.12 dataclass compat.")
else:
    print("  (no files needed patching)")

# Flush ALL cached esm modules so patched files are loaded fresh
_to_remove = [k for k in list(sys.modules.keys()) if k == "esm" or k.startswith("esm.")]
for _k in _to_remove:
    del sys.modules[_k]

import esm

print("⏳ Loading ESMFold model…")
base_model = esm.pretrained.esmfold_v1()
base_model = base_model.eval().to(DEVICE)
base_model.set_chunk_size(128)
print(f"✅ ESMFold loaded on {DEVICE}.")

  (no files needed patching)
⏳ Loading ESMFold model…


## 5 · Baseline ESMFold prediction

In [ ]:
import time

t0 = time.time()
print("⏳ Running ESMFold baseline prediction…")

with torch.no_grad():
    baseline_output = base_model.infer(SEQUENCE)

baseline_pdb   = base_model.output_to_pdb(baseline_output)[0]
baseline_plddt = baseline_output["mean_plddt"].item()

print(f"✅ Baseline pLDDT: {baseline_plddt:.2f}  ({time.time()-t0:.1f}s)")

## 6 · ProteinTTT — test-time training + prediction

In [ ]:
from proteinttt.models.esmfold import ESMFoldTTT, DEFAULT_ESMFOLD_TTT_CFG

# Configure TTT
ttt_cfg = DEFAULT_ESMFOLD_TTT_CFG
ttt_cfg.steps      = TTT_STEPS
ttt_cfg.lr         = LEARNING_RATE
ttt_cfg.batch_size = BATCH_SIZE
ttt_cfg.lora_rank  = LORA_RANK
ttt_cfg.lora_alpha = LORA_ALPHA
ttt_cfg.seed       = 0
ttt_cfg.eval_each_step = True   # avoid upstream UnboundLocalError

ttt_model = ESMFoldTTT.ttt_from_pretrained(
    base_model,
    ttt_cfg=ttt_cfg,
    esmfold_config=base_model.cfg,
)

# ── Run test-time training ──
t0 = time.time()
print(f"⏳ Running ProteinTTT ({TTT_STEPS} steps, lr={LEARNING_RATE}, batch={BATCH_SIZE}, "
      f"LoRA r={LORA_RANK}/α={LORA_ALPHA})…")

ttt_result = ttt_model.ttt(SEQUENCE)

print(f"  TTT finished in {time.time()-t0:.1f}s")

# ── Run prediction with TTT-adapted model ──
print("⏳ Running ESMFold+ProteinTTT prediction…")
with torch.no_grad():
    ttt_output = ttt_model.infer(SEQUENCE)

ttt_pdb   = ttt_model.output_to_pdb(ttt_output)[0]
ttt_plddt = ttt_output["mean_plddt"].item()

print(f"✅ ProteinTTT pLDDT: {ttt_plddt:.2f}")

# Clean up TTT state
ttt_model.ttt_reset()
del ttt_model
torch.cuda.empty_cache()

## 7 · Save PDB files

In [ ]:
baseline_pdb_path = os.path.join(OUTPUT_DIR, "esmfold_baseline.pdb")
ttt_pdb_path      = os.path.join(OUTPUT_DIR, "esmfold_proteinttt.pdb")

with open(baseline_pdb_path, "w") as f:
    f.write(baseline_pdb)
with open(ttt_pdb_path, "w") as f:
    f.write(ttt_pdb)

print(f"✅ Baseline PDB saved → {baseline_pdb_path}")
print(f"✅ TTT PDB saved      → {ttt_pdb_path}")

# Offer download in Colab
try:
    from google.colab import files as colab_files
    print("\n📥 Click below to download the PDB files:")
    colab_files.download(baseline_pdb_path)
    colab_files.download(ttt_pdb_path)
except ImportError:
    print("(Not running in Colab — files saved to disk.)")

## 8 · Results & 3D Visualisation

In [ ]:
from IPython.display import display, HTML
import html as html_lib, base64

# ── Summary table ──
improvement = ttt_plddt / baseline_plddt if baseline_plddt > 0 else 0

summary_html = f"""
<div style="font-family: 'Helvetica Neue', Arial, sans-serif; max-width: 700px;">
  <h2 style="margin-bottom: 4px;">Results</h2>
  <table style="border-collapse: collapse; width: 100%; font-size: 15px;">
    <tr style="background: #f7f7f7;">
      <th style="text-align:left; padding:8px 12px; border-bottom:2px solid #ddd;">Metric</th>
      <th style="text-align:right; padding:8px 12px; border-bottom:2px solid #ddd;">Value</th>
    </tr>
    <tr>
      <td style="padding:8px 12px; border-bottom:1px solid #eee;">Sequence length</td>
      <td style="padding:8px 12px; border-bottom:1px solid #eee; text-align:right;">{len(SEQUENCE)} residues</td>
    </tr>
    <tr>
      <td style="padding:8px 12px; border-bottom:1px solid #eee;">ESMFold pLDDT</td>
      <td style="padding:8px 12px; border-bottom:1px solid #eee; text-align:right; font-weight:600;">{baseline_plddt:.2f}</td>
    </tr>
    <tr>
      <td style="padding:8px 12px; border-bottom:1px solid #eee;">ESMFold + ProteinTTT pLDDT</td>
      <td style="padding:8px 12px; border-bottom:1px solid #eee; text-align:right; font-weight:600; color:#16a34a;">{ttt_plddt:.2f}</td>
    </tr>
    <tr>
      <td style="padding:8px 12px; border-bottom:1px solid #eee;">Improvement ratio</td>
      <td style="padding:8px 12px; border-bottom:1px solid #eee; text-align:right; font-weight:600;">{improvement:.2f}x</td>
    </tr>
    <tr>
      <td style="padding:8px 12px;">TTT config</td>
      <td style="padding:8px 12px; text-align:right; font-size:13px;">steps={TTT_STEPS}, lr={LEARNING_RATE}, batch={BATCH_SIZE}, LoRA r={LORA_RANK}/α={LORA_ALPHA}</td>
    </tr>
  </table>
</div>
"""
display(HTML(summary_html))


# ── 3Dmol.js viewer builder ──
def _mol_viewer(pdb_content, title):
    pdb_esc = pdb_content.replace("\\", "\\\\").replace("`", "\\`").replace("$", "\\$")
    title_esc = html_lib.escape(title)
    h = f"""
    <div style="font-family:sans-serif; font-weight:bold; text-align:center;
                padding:6px; background:#f0f0f0; font-size:14px;">{title_esc}</div>
    <div id="{id(pdb_content)}" style="width:100%; height:450px; position:relative;">
      <div style="position:absolute;right:10px;bottom:10px;z-index:10;background:rgba(255,255,255,.9);
                  border:1px solid #d9d9d9;border-radius:8px;padding:8px 10px;font-size:12px;
                  box-shadow:0 1px 4px rgba(0,0,0,.15);line-height:1.4;">
        <b>pLDDT</b><br>
        <span style="color:#0d57d3;">■</span> ≥90 &nbsp;
        <span style="color:#6acbf1;">■</span> 70–90 &nbsp;
        <span style="color:#fed936;">■</span> 50–70 &nbsp;
        <span style="color:#fd7d4d;">■</span> &lt;50
      </div>
    </div>
    <script src="https://3Dmol.csb.pitt.edu/build/3Dmol-min.js"></script>
    <script>
      (function() {{
        var v = $3Dmol.createViewer("{id(pdb_content)}", {{backgroundColor:"white"}});
        v.addModel(`{pdb_esc}`, "pdb");
        v.setStyle({{}}, {{cartoon:{{colorfunc:function(a){{
          if(a.b<50) return "#fd7d4d";
          if(a.b<70) return "#fed936";
          if(a.b<90) return "#6acbf1";
          return "#0d57d3";
        }}}}}});
        v.zoomTo(); v.render();
      }})();
    </script>"""
    return h


print("─" * 60)
print("ESMFold  (baseline)")
print("─" * 60)
display(HTML(_mol_viewer(baseline_pdb, f"ESMFold  —  pLDDT {baseline_plddt:.1f}")))

print()
print("─" * 60)
print("ESMFold + ProteinTTT")
print("─" * 60)
display(HTML(_mol_viewer(ttt_pdb, f"ESMFold + ProteinTTT  —  pLDDT {ttt_plddt:.1f}")))